# 🧠 Fine-tuning Llama-3.2-3B-Instruct for Mental Health Support

This notebook demonstrates fine-tuning **Llama-3.2-3B-Instruct** using **LoRA adapters** on the **ShivomH/MentalHealth-Support** dataset.

## 📋 Prerequisites
- GPU: RTX A6000 (48GB) / RTX 6000 Ada (48GB) / RTX 3090 (24GB)
- CUDA 11.8+ and PyTorch 2.0+
- Hugging Face account for model access
- Weights & Biases account for logging

## 1. 🔧 Setup and Installation

In [ ]:
# Install required packages
!pip install -U trl transformers accelerate peft datasets bitsandbytes trl wandb scipy sentencepiece protobuf

In [ ]:
# Import libraries
import os
import json
import torch
import wandb
import warnings
import numpy as np
from typing import Dict, List, Optional
from dataclasses import dataclass, field
from datetime import datetime

# Hugging Face imports
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from datasets import load_dataset, Dataset
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)
# from trl import SFTTrainer
from trl import SFTTrainer
from accelerate import Accelerator

warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. 🔍 GPU Memory Check

In [ ]:
def print_gpu_memory():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        
        print(f"GPU Memory Status:")
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        print(f"  Total: {total:.2f} GB")
        print(f"  Free: {total - allocated:.2f} GB")
    else:
        print("No GPU available")

# Initial GPU status
print_gpu_memory()
!nvidia-smi

## 3. ⚙️ Configuration

In [ ]:
@dataclass
class Config:
    """Training configuration"""
    
    # Model & Dataset
    model_name: str = "meta-llama/Llama-3.2-3B-Instruct"
    dataset_name: str = "enter_dataset_name"
    
    # Paths (replace with your chosen directory paths)
    output_dir: str = "directory_name_checkpoints" #Checkpoints will be stored here.
    adapter_dir: str = "directory_name_LoRA" #Final LoRA adapter will be stored here.
    merged_model_dir: str = "directory_name_merged" #Merged model will be stored here.
    cache_dir: str = "./cache"
    
    # LoRA Configuration
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.05
    target_modules: List[str] = field(default_factory=lambda: [
        "q_proj", "v_proj", "k_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ])
    
    # Training Parameters
    num_train_epochs: int = 4
    per_device_train_batch_size: int = 8
    per_device_eval_batch_size: int = 8
    gradient_accumulation_steps: int = 4
    learning_rate: float = 3e-5
    warmup_ratio: float = 0.05
    max_seq_length: int = 1024
    
    # Optimization
    optim: str = "paged_adamw_32bit"
    lr_scheduler_type: str = "cosine"
    gradient_checkpointing: bool = True
    bf16: bool = True
    tf32: bool = True
    
    # Logging
    logging_steps: int = 50
    save_steps: int = 400
    eval_steps: int = 200
    save_total_limit: int = 3
    
    # W&B
    use_wandb: bool = True
    wandb_project: str = "your_project_name_in_wandb"
    wandb_run_name: str = f"llama3.2-lora-{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # Hub
    push_to_hub: bool = False
    hub_model_id: str = "your_hub_model_id_to_push_on_huggingface"
    hub_private: bool = True
    
config = Config()
print("Configuration loaded successfully!")

## 4. 🔐 Authentication

In [ ]:
# Hugging Face login (required for Llama models)
from huggingface_hub import login

# You need to accept the Llama license agreement first:
# https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

# Option 1: Use token from environment
os.environ["HF_TOKEN"] = "your_huggingface_token"
login(token=os.environ["HF_TOKEN"])

# Option 2: Interactive login
# login()

# W&B login
if config.use_wandb:
    wandb.login()
    wandb.init(
        project=config.wandb_project,
        name=config.wandb_run_name,
        config=config.__dict__
    )

## 5. 📊 Dataset Loading and Preparation

In [ ]:
# Load dataset
print(f"Loading dataset: {config.dataset_name}")
dataset = load_dataset(config.dataset_name, split="train")

# Dataset statistics
print(f"\nDataset size: {len(dataset)} examples")
print(f"Dataset features: {dataset.features}")
print(f"\nFirst example:")
print(json.dumps(dataset[0], indent=2)[:1000] + "...")

In [ ]:
# Split into train and validation
dataset_split = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"Training examples: {len(train_dataset):,}")
print(f"Validation examples: {len(eval_dataset):,}")

# Analyze conversation types
def analyze_conversations(dataset):
    multi_turn = 0
    single_turn = 0
    
    for example in dataset:
        messages = example["messages"]
        user_messages = [m for m in messages if m["role"] == "user"]
        if len(user_messages) > 1:
            multi_turn += 1
        else:
            single_turn += 1
    
    return multi_turn, single_turn

multi, single = analyze_conversations(train_dataset)
print(f"\nConversation types in training set:")
print(f"  Multi-turn: {multi:,}")
print(f"  Single-turn: {single:,}")

## 6. 🤖 Model and Tokenizer Setup

In [ ]:
# Load tokenizer
print(f"Loading tokenizer: {config.model_name}")
tokenizer = AutoTokenizer.from_pretrained(
    config.model_name,
    cache_dir=config.cache_dir,
    use_fast=True,
    trust_remote_code=True
)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Tokenizer loaded successfully")
print(f"  Vocabulary size: {len(tokenizer)}")
print(f"  Pad token: {tokenizer.pad_token}")
print(f"  EOS token: {tokenizer.eos_token}")

### **Important Note: The process should be followed in correct sequence, else the model will not be loaded correctly.

#### Load the quantized model -> Prepare model for k-bit training -> Gradient Checkpointing -> Apply LoRA to the model

In [ ]:
# Quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model
print(f"\nLoading model: {config.model_name}")
print("This may take a few minutes...")

model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    cache_dir=config.cache_dir,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Enable gradient checkpointing
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

print("Model loaded successfully!")
print_gpu_memory()

## 7. 🔧 LoRA Configuration

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=config.target_modules,
    lora_dropout=config.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params:,} || "
        f"All params: {all_params:,} || "
        f"Trainable %: {100 * trainable_params / all_params:.2f}%"
    )

print("\nLoRA Configuration:")
print(lora_config)
print("\nModel with LoRA:")
print_trainable_parameters(model)

## 8. 📝 Data Preprocessing

In [ ]:
def preprocess_function(examples):
    """
    Apply chat template and tokenize.
    Supports multi-turn and single-turn seamlessly.
    Returns lists (not tensors) for compatibility with HuggingFace datasets.
    """
    texts = []
    for messages in examples["messages"]:
        # Ensure it's always a list of dicts
        if isinstance(messages, dict):
            messages = [messages]

        # Apply chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    # Tokenize → return lists, not tensors
    model_inputs = tokenizer(
        texts,
        max_length=config.max_seq_length,
        truncation=True,
        padding="max_length",
        add_special_tokens=False,
    )

    # Labels = input_ids
    labels = model_inputs["input_ids"].copy()

    # Mask padding token positions (-100 tells HF Trainer to ignore them in loss)
    labels = [
        [(tok if tok != tokenizer.pad_token_id else -100) for tok in seq]
        for seq in labels
    ]
    model_inputs["labels"] = labels

    return model_inputs

# Test preprocessing on a sample
sample = train_dataset.select(range(1))
sample_processed = preprocess_function(sample)
print("Sample preprocessing successful!")
print(f"Input length: {len(sample_processed['input_ids'][0])}")
print(f"Attention mask length: {len(sample_processed['attention_mask'][0])}")

In [ ]:
train_dataset_proc = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

eval_dataset_proc = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
)

## 9. 🎯 Training Setup

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=config.output_dir,
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    warmup_ratio=config.warmup_ratio,
    lr_scheduler_type=config.lr_scheduler_type,
    
    # Optimization
    optim=config.optim,
    bf16=config.bf16,
    tf32=config.tf32,
    gradient_checkpointing=config.gradient_checkpointing,
    
    # Logging
    logging_dir=f"{config.output_dir}/logs",
    logging_steps=config.logging_steps,
    save_steps=config.save_steps,
    eval_steps=config.eval_steps,
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=config.save_total_limit,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # W&B
    report_to="wandb" if config.use_wandb else "none",
    run_name=config.wandb_run_name if config.use_wandb else None,
    
    # Hub
    push_to_hub=config.push_to_hub,
    hub_model_id=config.hub_model_id if config.push_to_hub else None,
    hub_private_repo=config.hub_private if config.push_to_hub else None,
    
    # Other
    remove_unused_columns=False,
    label_names=["labels"],
    seed=42,
    dataloader_num_workers=4,
    group_by_length=True,
)

print("Training arguments configured")
print(f"\nEffective batch size: {config.per_device_train_batch_size * config.gradient_accumulation_steps}")
print(f"Total optimization steps: ~{len(train_dataset) // (config.per_device_train_batch_size * config.gradient_accumulation_steps) * config.num_train_epochs}")

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_proc,
    eval_dataset=eval_dataset_proc,
    tokenizer=tokenizer,   
)

## 10. 🚀 Training

In [ ]:
# Start training
print("Starting training...")
print("="*50)

try:
    # Train
    trainer.train()
    
    # Save final model
    print("\nSaving final model...")
    trainer.save_model(config.output_dir)
    
    # Save metrics
    with open(f"{config.output_dir}/training_metrics.json", "w") as f:
        json.dump(trainer.state.log_history, f, indent=2)
    
    print("Training completed successfully!")
    
except KeyboardInterrupt:
    print("\nTraining interrupted by user")
    trainer.save_model(f"{config.output_dir}/checkpoint-interrupted")
    
finally:
    print_gpu_memory()

In [ ]:
print("=== RAW EXAMPLE ===")
raw_sample = train_dataset[0]   # first unprocessed example
print(json.dumps(raw_sample, indent=2)[:1000])  # pretty print

print("\n=== CHAT TEMPLATE EXAMPLE ===")
messages = raw_sample["messages"]
templated = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False
)
print(templated)

print("\n=== TOKENIZED EXAMPLE ===")
proc_sample = train_dataset_proc[0]  # already preprocessed by map
for k, v in proc_sample.items():
    print(f"{k}: {len(v)} -> {v[:20]}")  # print first 20 tokens

print("\n=== DECODED FROM IDS ===")
decoded = tokenizer.decode(proc_sample["input_ids"], skip_special_tokens=False)
print(decoded)

## 11. 📊 Evaluation

In [ ]:
# Evaluate on validation set
print("Evaluating model on validation set...")
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

# Calculate perplexity
import math
perplexity = math.exp(eval_results['eval_loss'])
print(f"\nPerplexity: {perplexity:.2f}")

In [20]:
def evaluate_generation(model, tokenizer, prompts, max_new_tokens=256):
    """
    Evaluate model generation on sample prompts.
    """
    model.eval()
    results = []
    
    for prompt_data in prompts:
        messages = prompt_data["messages"]
        
        # Apply chat template
        input_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        # Tokenize
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=config.max_seq_length
        ).to(model.device)
        
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        # Decode
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the assistant's response
        response = generated_text.split("assistant")[-1].strip()
        
        results.append({
            "input": messages,
            "response": response
        })
    
    return results

# Test prompts
test_prompts = [
    # Single-turn
    {
        "messages": [
            {"role": "system", "content": "You are a compassionate mental health support assistant."},
            {"role": "user", "content": "I've been feeling really anxious about my upcoming job interview. Any advice?"}
        ]
    },
    # Multi-turn
    {
        "messages": [
            {"role": "system", "content": "You are a compassionate mental health support assistant."},
            {"role": "user", "content": "I'm having trouble sleeping lately."},
            {"role": "assistant", "content": "I understand that sleep issues can be really frustrating. Can you tell me more about what's been happening? When did this start, and have you noticed any patterns?"},
            {"role": "user", "content": "It started about two weeks ago. I keep waking up at 3 AM and can't fall back asleep."}
        ]
    }
]

# Evaluate
print("Generating responses for test prompts...\n")
results = evaluate_generation(model, tokenizer, test_prompts)

for i, result in enumerate(results, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}:")
    print(f"\nUser: {result['input'][-1]['content']}")
    print(f"\nAssistant: {result['response']}")
    print(f"{'='*60}")

Generating responses for test prompts...


Test 1:

User: I've been feeling really anxious about my upcoming job interview. Any advice?

Assistant: I'm sorry to hear that you're feeling anxious about your job interview. It's completely normal to feel that way before a big event like this. Can you tell me a bit more about what specifically is causing your anxiety? Is it the fear of rejection or something else? And what are your goals for this job interview? Understanding your concerns will help me provide you with the best guidance.

Test 2:

User: It started about two weeks ago. I keep waking up at 3 AM and can't fall back asleep.

Assistant: I'm sorry to hear that you're experiencing disrupted sleep patterns. Have you tried any techniques to help you fall back asleep when you wake up in the middle of the night? Perhaps a warm bath or deep breathing exercises? Have you noticed anything that might be triggering your wakefulness?


## 12. 💾 Save LoRA Adapter

In [ ]:
# Save LoRA adapter only
print(f"Saving LoRA adapter to {config.adapter_dir}")
model.save_pretrained(config.adapter_dir)
tokenizer.save_pretrained(config.adapter_dir)

# Save configuration
with open(f"{config.adapter_dir}/training_config.json", "w") as f:
    json.dump(config.__dict__, f, indent=2, default=str)

print("LoRA adapter saved successfully!")

# Check adapter size
import os
total_size = 0
for root, dirs, files in os.walk(config.adapter_dir):
    for file in files:
        if file.endswith('.bin') or file.endswith('.safetensors'):
            file_path = os.path.join(root, file)
            total_size += os.path.getsize(file_path)

print(f"\nAdapter size: {total_size / 1024**2:.2f} MB")

## 13. 🔀 Merge LoRA with Base Model (Optional)

In [ ]:
# Option to merge LoRA weights with base model for easier deployment
merge_model = input("Do you want to merge LoRA with base model? (y/n): ").lower() == 'y'

if merge_model:
    print("\nMerging LoRA adapter with base model...")
    print("This will require loading the full model in memory.")
    
    # Clear GPU memory first
    import gc
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    # Load base model without quantization
    from peft import PeftModel
    
    base_model = AutoModelForCausalLM.from_pretrained(
        config.model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        cache_dir=config.cache_dir,
    )
    
    # Load LoRA weights
    model_with_lora = PeftModel.from_pretrained(base_model, config.adapter_dir)
    
    # Merge and unload
    merged_model = model_with_lora.merge_and_unload()
    
    # Save merged model
    print(f"Saving merged model to {config.merged_model_dir}")
    merged_model.save_pretrained(config.merged_model_dir)
    tokenizer.save_pretrained(config.merged_model_dir)
    
    print("Merged model saved successfully!")
    
    # Cleanup
    del base_model, model_with_lora, merged_model
    gc.collect()
    torch.cuda.empty_cache()

## 14. 🤗 Push to Hub (Optional)

In [ ]:
# Push to Hugging Face Hub
if config.push_to_hub:
    print(f"Pushing model to Hub: {config.hub_model_id}")
    
    # Push adapter
    from huggingface_hub import HfApi
    api = HfApi()
    
    # Create model card
    model_card = f"""
---
library_name: peft
base_model: {config.model_name}
tags:
- mental-health
- llama
- lora
- conversational
datasets:
- {config.dataset_name}
---

# Llama-3.2-3B Mental Health Support LoRA

This is a LoRA adapter for {config.model_name} fine-tuned on mental health support conversations.

## Training Details
- **Base Model**: {config.model_name}
- **Dataset**: {config.dataset_name}
- **LoRA Rank**: {config.lora_r}
- **LoRA Alpha**: {config.lora_alpha}
- **Learning Rate**: {config.learning_rate}
- **Epochs**: {config.num_train_epochs}
- **Batch Size**: {config.per_device_train_batch_size * config.gradient_accumulation_steps}

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model and tokenizer
model = AutoModelForCausalLM.from_pretrained("{config.model_name}")
tokenizer = AutoTokenizer.from_pretrained("{config.model_name}")

# Load LoRA adapter
model = PeftModel.from_pretrained(model, "{config.hub_model_id}")
```

## Disclaimer
This model is for research purposes only. It should not replace professional mental health services.
    """
    
    with open(f"{config.adapter_dir}/README.md", "w") as f:
        f.write(model_card)
    
    # Push to hub
    api.upload_folder(
        folder_path=config.adapter_dir,
        repo_id=config.hub_model_id,
        repo_type="model",
        private=config.hub_private
    )
    
    print(f"Model pushed to: https://huggingface.co/{config.hub_model_id}")

## 15. 🎮 Inference

In [ ]:
# Load fine-tuned model for inference
print("Loading fine-tuned model for inference...")

# Clear memory if needed
import gc
if 'model' in locals():
    del model
gc.collect()
torch.cuda.empty_cache()

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Paths to your saved model/tokenizer
model_path = "merged_model_directory_path"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model (quantization if needed)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
def generate_response(
    model,
    tokenizer,
    messages,
    max_new_tokens=256,
    temperature=0.4,
    top_p=0.8,
    do_sample=True
):
    """
    Generate response from the model given messages.
    
    Args:
        messages: List of message dicts with 'role' and 'content'
    
    Returns:
        Generated response string
    """
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=config.max_seq_length
    ).to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # Decode
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract assistant response
    response = generated.split("assistant\n")[-1].strip()
    
    return response

print("Inference function ready!")

In [51]:
# Interactive chat interface
print("="*60)
print("💬 Mental Health Support Chat")
print("Type 'quit' to exit, 'reset' to start a new conversation")
print("="*60)

# System prompt
system_message = {
    "role": "system",
    "content": """You are a compassionate and supportive mental health assistant. 
    Provide helpful, empathetic responses while maintaining appropriate boundaries. 
    Never reveal personal info like names, phone numbers, or addresses.
    Remember: You are not a replacement for professional mental health services."""
}

# Conversation history
conversation = [system_message]

while True:
    # Get user input
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() == 'quit':
        print("Goodbye! Take care.")
        break
    
    if user_input.lower() == 'reset':
        conversation = [system_message]
        print("Conversation reset.")
        continue
    
    if not user_input:
        continue
    
    # Add user message
    conversation.append({"role": "user", "content": user_input})
    
    # Generate assistant reply
    reply = generate_response(model, tokenizer, conversation)
    
    # Add assistant message to history
    conversation.append({"role": "assistant", "content": reply})
    
    # Print assistant response
    print(f"\nAssistant: {reply}")

💬 Mental Health Support Chat
Type 'quit' to exit, 'reset' to start a new conversation



You:  Can you diagnose me with depression? I think I have it.



Assistant: I'm here to listen and support you. It takes a lot of courage to reach out and acknowledge that you may be experiencing depression. It's important to remember that I'm not a medical professional, so I can't diagnose you. However, I can offer guidance and support as you explore your feelings and thoughts.

When you say you think you have depression, what are some of the symptoms you've been experiencing? Can you tell me more about what you've been going through? This will help us understand your situation better. Is there anything specific that has triggered these feelings? And how long have you been feeling this way? Understanding the context can be helpful in finding ways to cope. Let's take it at your pace and work together to find some solutions. What do you think would be helpful for you right now?



You:  quit


Goodbye! Take care.
